# Choosing your plotting backend

DiffeRT supports various backends for plotting scenes and related-objects:

- [VisPy](https://vispy.org/), a high-performance interactive visualization library that leverages the
  power of modern GPUs through OpenGL. Installing VisPy on its own is not sufficient and you will need at least one
  of the VisPy backends to be installed: https://vispy.org/installation.html.
  If available, VisPy will be used by default as it provides the best user experience, especially on large scenes.
- [Matplotlib](https://matplotlib.org/), a very user-friendly and popular tool for creating plots in Python.
- [Plotly](https://plotly.com/python/), a Python library for building beautiful interactive plots for the web.

Additionally, all those backends can be used within Jupyter notebooks.
For VisPy, you will need [`jupyter_rfb`](https://github.com/vispy/jupyter_rfb) and a recent JupyterLab installation,
that you can obtain with `pip install differt[jupyter]`.

In [ ]:
from pathlib import Path

from differt.geometry import TriangleMesh

mesh_file = Path("bruxelles.obj")
mesh = TriangleMesh.load_obj(mesh_file)

## VisPy

VisPy is the default plotting backend, and supports all plotting utilities.

Because this is the default backend, you do not have to specify it,
[unless you changed the default backend](change-default-backend).

(vispy-pros)=
### Pros

* Performances;
* Has a relatively good documentation;
* Integrates well with Qt applications;
* Implements all plotting utilities.

(vispy-cons)=
### Cons

* Does not support interactive offline rendering (i.e., a static snapshot is shown);
* Requires an additional and possibly large dependency (e.g., PyQt6).

In [ ]:
canvas = mesh.plot(backend="vispy")
canvas

## Matplotlib

Matplotlib is probably the most-used Python plotting library.

(matplotlib-pros)=
### Pros

* Has excellent online documentation;
* Can be integrated with Qt applications;
* Support a lot of configuration options and output formats;
* Simple and user-friendly API.

(matplolib-cons)=
### Cons

* Does not support interactive offline rendering (i.e., a static snapshot is shown);
* Poor 3D graphics support;
* Interactive and 3D plots are slow.

In [ ]:
%matplotlib widget

mesh.plot(backend="matplotlib", alpha=0.5, shade=True, linewidth=0.2, antialiased=True)

## Plotly

(plotly-pros)=
### Pros

* Has excellent interactive mode;
* Provides beautiful plots with almost zero-configuration needed;
* Supports interactive offline rendering (i.e., plots can be fully contained in HTML pages).

(plotly-cons)=
### Cons

* Has a no-so-good online documentation (i.e., using functions that do not have usage examples is quite hard);
* Does not support interactive offline rendering (i.e., a static snapshot is shown);
* Slow 3D graphics and relatively *glitchy*.

In [ ]:
mesh.plot(backend="plotly", opacity=0.5)

(change-default-backend)=
## Changing the default backend

When not specified, the `backend` parameter defaults to `None`, which in turn indicates
that you will use the default backend. So `mesh.plot()`
is equivalent to `mesh.plot(backend="vispy")`.

If you want to use another backend by default, you can do so by importing `differt.plotting` and
calling `use` with the desired backend name.

In [ ]:
import differt.plotting as dplt  # noqa: E402

dplt.use("plotly")

mesh.plot()